In [3]:
from graphviz import Digraph
import torch
from torch.autograd import Variable


def make_dot(var, params=None):
    """ Produces Graphviz representation of PyTorch autograd graph
    Blue nodes are the Variables that require grad, orange are Tensors
    saved for backward in torch.autograd.Function
    Args:
        var: output Variable
        params: dict of (name, Variable) to add names to node that
            require grad (TODO: make optional)
    """
    if params is not None:
        assert isinstance(params.values()[0], Variable)
        param_map = {id(v): k for k, v in params.items()}

    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()

    def size_to_str(size):
        return '('+(', ').join(['%d' % v for v in size])+')'

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                name = param_map[id(u)] if params is not None else ''
                node_name = '%s\n %s' % (name, size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)
    add_nodes(var.grad_fn)
    return dot

In [5]:
import colorplot as cp
net, train_producer = cp._setup(
    label_path   = 'data/raw/training2017/REFERENCE.csv',
    data_path    = 'data/raw/training2017/',
    weights_path = 'dry/saved/EncdedWideResnetFIXED/state_dict_highscore'
)
g = make_dot(net)
g.view()

AttributeError: 'CombinedTransform' object has no attribute 'grad_fn'

In [6]:
net

CombinedTransform (
  (models): ModuleList (
    (0): SkipFCN (
      (pool): MaxPool1d (size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (activation): SELU (
      )
      (conv1): Conv1d(16, 16, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
      (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv1d(16, 16, kernel_size=(9,), stride=(1,), padding=(8,), dilation=(2,), bias=False)
      (bn2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv1d(16, 32, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
      (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True)
      (conv4): Conv1d(32, 32, kernel_size=(9,), stride=(1,), padding=(8,), dilation=(2,), bias=False)
      (bn4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True)
      (conv5): Conv1d(48, 64, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
      (bn5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv6): Co

In [2]:
import torch as th
params = th.load('dry/saved/EncdedWideResnetFIXED/state_dict_highscore')

# convert numpy arrays to torch Variables
for k in sorted(params.keys()):
    v = params[k]
    print(k, v.shape)
    params[k] = Variable(torch.from_numpy(v), requires_grad=True)

classifier.0.bias torch.Size([256])


RuntimeError: from_numpy expects an np.ndarray but got torch.cuda.FloatTensor

In [3]:
def define_model(params):
    def conv2d(input, params, base, stride=1, pad=0):
        return F.conv2d(input, params[base + '.weight'],
                        params[base + '.bias'], stride, pad)

    def group(input, params, base, stride, n):
        o = input
        for i in range(0,n):
            b_base = ('%s.block%d.conv') % (base, i)
            x = o
            o = conv2d(x, params, b_base + '0', pad=1, stride=i==0 and stride or 1)
            o = F.relu(o)
            o = conv2d(o, params, b_base + '1', pad=1)
            if i == 0 and stride != 1:
                o += F.conv2d(x, params[b_base + '_dim.weight'], stride=stride)
            else:
                o += x
            o = F.relu(o)
        return o
    
    # determine network size by parameters
    blocks = [sum([re.match('group%d.block\d+.conv0.weight'%j, k) is not None
                   for k in params.keys()]) for j in range(4)]

    def f(input, params):
        o = F.conv2d(input, params['conv0.weight'], params['conv0.bias'], 2, 3)
        o = F.relu(o)
        o = F.max_pool2d(o, 3, 2, 1)
        o_g0 = group(o, params, 'group0', 1, blocks[0])
        o_g1 = group(o_g0, params, 'group1', 2, blocks[1])
        o_g2 = group(o_g1, params, 'group2', 2, blocks[2])
        o_g3 = group(o_g2, params, 'group3', 2, blocks[3])
        o = F.avg_pool2d(o_g3, 7, 1, 0)
        o = o.view(o.size(0), -1)
        o = F.linear(o, params['fc.weight'], params['fc.bias'])
        return o
    return f

f = define_model(params)

NameError: name 'params' is not defined